In [ ]:
# pip install folium

In [ ]:
import pulp
from pulp import *

import numpy as np
import pandas as pd

import datetime
import matplotlib.pyplot as plt

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
import folium
import requests

## Load Data

In [ ]:
file_path = 'CSV_Files_Food_Delivery/'

In [ ]:
df_distances = pd.read_csv(file_path + 'distances.csv')

df_ordersA = pd.read_csv(file_path + 'part1_ordersA.csv')
df_ordersB = pd.read_csv(file_path + 'part1_ordersB.csv')

In [ ]:
df_regions = pd.read_csv(file_path + 'regions.csv')

In [ ]:
df_distances.head()

In [ ]:
df_ordersA

In [ ]:
df_ordersB

# Part I

## Sets

In [ ]:
starting_location = 'Downtown Toronto (Rosedale)'

In [ ]:
df = df_ordersB

In [ ]:
# Get start and end locations
start_locations = list(df['restaurant'].unique()) + list(df['customer'].unique())
end_locations = start_locations.copy()

start_locations = list(dict.fromkeys(start_locations))
end_locations = list(dict.fromkeys(end_locations))

if starting_location in start_locations:
    start_locations.remove(starting_location)
    start_locations.append(starting_location)
else:
    start_locations.append(starting_location)

In [ ]:
# Get unique Restraurants and Customers
restaurants = df['restaurant'].unique().tolist()
customers = df['customer'].unique().tolist()

In [ ]:
num_steps = np.arange(1, len(end_locations)+1).tolist()

In [ ]:
end_locations

In [ ]:
num_steps

In [ ]:
order_df = pd.DataFrame({'Restaurant':[], 'Customer':[]})

for index, row in df.iterrows():
    order_df.loc[index, 'Restaurant'] = row["restaurant"]
    order_df.loc[index, 'Customer'] = row["customer"]

order_df

In [ ]:
travel_df = pd.DataFrame({'Start':[], 'End': [], 'Distance': [], 'Time': []})


for i in start_locations:
    for j in end_locations: 
        if i == j:
            new_row_data = {'Start': i, 'End': j, 'Distance': 0, 'Time': np.nan}
            new_row = pd.DataFrame(new_row_data, index=[0])
            travel_df = pd.concat([travel_df, new_row], ignore_index=True)
        else:
            d_temp = float(df_distances[(df_distances['origin'] == i) & 
                                                          (df_distances['destination'] == j)]['distance'])
            new_row_data = {'Start': i, 'End': j, 'Distance': d_temp, 'Time': np.nan}
            new_row = pd.DataFrame(new_row_data, index=[0])
            travel_df = pd.concat([travel_df, new_row], ignore_index=True)

In [ ]:
travel_df

## Model Setup

In [ ]:
model = LpProblem(name = 'Part1_Model', sense = LpMinimize)

xVar = LpVariable.dict('x', (start_locations, end_locations, num_steps), cat = LpBinary)

In [ ]:
obj = lpSum([travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0] * xVar[(i,j,t)] \
             for i in start_locations for j in end_locations for t in num_steps])
model += obj

## Constraints

In [ ]:
def restaurant_customer_match(customer, order_df):
    for index, row in order_df.iterrows():
        if customer == order_df.loc[index, 'Customer']:
            return order_df.loc[index, 'Restaurant']

In [ ]:
# Every location receives order once
for j in end_locations: 
    model += lpSum([xVar[(i,j,t)] for i in start_locations for t in num_steps]) == 1

In [ ]:
# Convervation of flow
for t in num_steps[:-1]:
    for j in end_locations:
        model += (lpSum([xVar[(i,j,t)] for i in start_locations]) == lpSum([xVar[(j,k,t+1)] for k in end_locations]))

In [ ]:
# First node has an outflow of 1, others 0
model += lpSum([xVar[(start_locations[-1],j,1)] for j in restaurants]) == 1

for i in start_locations[:-1]:
    model += lpSum([xVar[(i,j,1)] for j in end_locations]) == 0

In [ ]:
# Visit restaurant before customer
for t in num_steps: 
    for j in end_locations: 
        if j in customers: 
            model += pulp.lpSum( [xVar[i,j,t] for i in start_locations]) <= pulp.lpSum([xVar[i, restaurant_customer_match(j, order_df),t_] \
                                                                                        for i in start_locations \
                                                                                        for t_ in num_steps[:t]])

## Solve

In [ ]:
# Solve the model
model.solve()
print("Status:", LpStatus[model.status])

In [ ]:
# Total Distance
total_distance = pulp.value(model.objective)
print("Total Distance: " , total_distance)

In [ ]:
# Print the results
path=[]
for t in num_steps:
    print(f'step:{t}\n')
    for i in start_locations:
         for j in end_locations:
                if (xVar[(i,j,t)].varValue == 1) :
                    print(f"TRAVEL FROM {i}  TO {j}")
                    
                    if i not in path:
                        path.append(i)
                    if j not in path:
                        path.append(j)

In [ ]:
route_=[]
for index,r in enumerate(path):
    lat=df_regions.loc[df_regions['name'] == r]['latitude'].values[0]
    log=df_regions.loc[df_regions['name'] == r]['longitude'].values[0]
    route_.append((lat,log))

In [ ]:
# Load the GeoJSON data for Toronto from a URL
url = 'https://raw.githubusercontent.com/jasonicarter/toronto-geojson/master/toronto_crs84.geojson'
response = requests.get(url)
toronto_geojson = response.json()

# Create a map of Toronto
m = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# Add the GeoJSON data to the map as a layer
folium.GeoJson(toronto_geojson).add_to(m)

# Define the route as a list of (lat, lon) coordinates
route = route_

# Add the route to the map as a PolyLine
folium.PolyLine(route, color='red').add_to(m)

# Add numbered markers for each location to the map
for i, location in enumerate(route):
    folium.Marker(location=location, icon=folium.Icon(color='blue'), 
                  popup=str(i+1)).add_to(m)

# Display the map
m


# Part II

## Load Data

In [ ]:
df_distances = pd.read_csv(file_path + 'distances.csv')

df_ordersA = pd.read_csv(file_path + 'part2_ordersA.csv')
df_ordersB = pd.read_csv(file_path + 'part2_ordersB.csv')

In [ ]:
df_distances.head()

In [ ]:
df_ordersA

In [ ]:
df_ordersB

## Data Preparation

In [ ]:
# Route A or B
route = 'B'

In [ ]:
starting_location = 'Downtown Toronto (Rosedale)'
average_velocity = 40 # Km/Hr
average_wait = 5 # Mins

# Convert to Km/Min
velocity_min = average_velocity / 60

In [ ]:
# Route variables
W = 120
test_W = range(0, 120, 5)

if route == 'A':
    print('Route A')
    df = df_ordersA
    start_time = pd.to_datetime('2022-04-02 19:00:00')
else:
    print('Route B')
    df = df_ordersB
    start_time = pd.to_datetime('2022-04-02 17:00:00')

In [ ]:
# Get start and end locations
start_locations = list(df['restaurant'].unique()) + list(df['customer'].unique())
end_locations = start_locations.copy()

start_locations = list(dict.fromkeys(start_locations))
end_locations = list(dict.fromkeys(end_locations))

if starting_location in start_locations:
    start_locations.remove(starting_location)
    start_locations.append(starting_location)
else:
    start_locations.append(starting_location)


In [ ]:
order_locations = list(df['restaurant']) + list(df['customer'])
order_locations

In [ ]:
# Calculate the number of stops 
num_steps = np.arange(1, len(end_locations)+1).tolist()

In [ ]:
# Get unique Restraurants and Customers
restaurants = df['restaurant'].unique().tolist()
customers = df['customer'].unique().tolist()

In [ ]:
order_df = pd.DataFrame({'Restaurant':[], 'Customer':[]})

for index, row in df.iterrows():
    order_df.loc[index, 'Restaurant'] = row["restaurant"]
    order_df.loc[index, 'Customer'] = row["customer"]

order_df

In [ ]:
travel_df = pd.DataFrame({'Start':[], 'End': [], 'Distance': [], 'Time': []})


for i in start_locations:
    for j in end_locations: 
        if i == j:
            new_row_data = {'Start': i, 'End': j, 'Distance': 0, 'Time': 0}
            new_row = pd.DataFrame(new_row_data, index=[0])
            travel_df = pd.concat([travel_df, new_row], ignore_index=True)
        else:
            d_temp = float(df_distances[(df_distances['origin'] == i) & 
                                                          (df_distances['destination'] == j)]['distance'])
            t_temp = d_temp/average_velocity * 60
            new_row_data = {'Start': i, 'End': j, 'Distance': d_temp, 'Time': t_temp}
            new_row = pd.DataFrame(new_row_data, index=[0])
            travel_df = pd.concat([travel_df, new_row], ignore_index=True)

travel_df.head()

In [ ]:
df['estimated availability'] = pd.to_datetime(df['estimated availability'])
df['Converted Availability'] = (df['estimated availability'] - df['estimated availability'].min()).dt.total_seconds() / 60

In [ ]:
df

## Model

In [ ]:
model = LpProblem(name = 'Part2_Model', sense = LpMinimize)

xVar = LpVariable.dict('x', (start_locations, end_locations, num_steps), cat = LpBinary)
# zVar = LpVariable.dict('z', num_steps, lowBound = 0.0, cat = LpContinuous)
dVar = LpVariable.dict('d', [0] + num_steps, lowBound = -9999.0, cat = LpContinuous)
aVar = LpVariable.dict('a', [0] + num_steps, lowBound = -9999.0, cat = LpContinuous)
wVar = LpVariable.dict('w', (start_locations, num_steps), lowBound = 0.0, cat = LpContinuous)
# vVar = LpVariable.dict('w', (start_locations, end_locations), lowBound = 0.0, cat = LpContinuous)

In [ ]:
obj = lpSum([travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0] * xVar[(i,j,t)] \
             for i in start_locations for j in end_locations for t in num_steps])
model += obj

In [ ]:
model += dVar[(0)] <= 0

In [ ]:
def travel_time_lookup(i, j, travel_df):
    return travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Time'].iloc[0]

In [ ]:
# # Step time calculation
# for t in num_steps:
#     model += dVar[(t)] == dVar[(t-1)] + lpSum([travel_time_lookup(i, j, travel_df) * xVar[(i,j,t)] \
#                                                for i in start_locations for j in restaurants]) + \
#                           lpSum([(travel_time_lookup(i, j, travel_df) + 5) * xVar[(i,j,t)] \
#                                  for i in start_locations for j in customers])

In [ ]:
# # Assign time to each travel in vVar
# for i in start_locations:
#     for j in end_locations:
#         model += vVar[(i,j)] == travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Time'].iloc[0]

In [ ]:
# # Driver arrives after the food is ready
# for t in num_steps:
#     for i in start_locations:
#         for j in range(0, int(len(order_locations)/2)):
#             model += dVar[(t)] >= df.loc[j, 'Converted Availability'] * xVar[(i,df.loc[j, 'restaurant'],t)]

In [ ]:
# Step time calculation
for t in num_steps:
    model += aVar[(t)] == dVar[(t-1)] + lpSum([travel_time_lookup(i, j, travel_df) * xVar[(i,j,t)] \
                                               for i in start_locations for j in restaurants]) + \
                          lpSum([(travel_time_lookup(i, j, travel_df) + 5) * xVar[(i,j,t)] \
                                 for i in start_locations for j in customers])
    model += aVar[(t)] <= dVar[(t)]

In [ ]:
# Driver leaves after the food is ready
for t in num_steps:
    for i in start_locations:
        for j in restaurants:
            model += dVar[(t)] >= df[df['restaurant'] == j]['Converted Availability'].iloc[0] * xVar[(i,j,t)]

In [ ]:
# Visit restaurant before customer
for t in num_steps: 
    for j in end_locations: 
        if j in customers: 
            model += pulp.lpSum( [xVar[i,j,t] for i in start_locations]) <= pulp.lpSum([xVar[i, restaurant_customer_match(j, order_df),t_] \
                                                                                        for i in start_locations \
                                                                                        for t_ in num_steps[:t]])

In [ ]:
for idx, row in df.iterrows():
    rst, cus, time = row['restaurant'], row['customer'], row['Converted Availability']
    for t in num_steps:
        model += wVar[(cus, t)] <= 10000000 * lpSum(xVar[(i, cus, t)] for i in start_locations if i != cus)
        
        model += wVar[(cus, t)] <= aVar[t] - time + 10000000 * (1 - lpSum(xVar[(i, cus, t)] for i in start_locations if i !=cus))
        model += wVar[(cus, t)] >= aVar[t] - time - 10000000 * (1 - lpSum(xVar[(i, cus, t)] for i in start_locations if i !=cus))

In [ ]:
model += W >= lpSum(wVar[(c,t)] for c in customers for t in num_steps)/len(df)

In [ ]:
# Every location receives order once
for j in end_locations: 
    model += lpSum([xVar[(i,j,t)] for i in start_locations for t in num_steps]) == 1

In [ ]:
# Conservation of flow
for t in num_steps[:-1]:
    for j in end_locations:
        model += (lpSum([xVar[(i,j,t)] for i in start_locations]) == lpSum([xVar[(j,k,t+1)] for k in end_locations]))

In [ ]:
# First node has an outflow of 1, others 0
model += lpSum([xVar[(start_locations[-1],j,1)] for j in restaurants]) == 1

for i in start_locations[:-1]:
    model += lpSum([xVar[(i,j,1)] for j in end_locations]) == 0

## Solve

In [ ]:
# Solve the model
model.solve()
print("Status:", LpStatus[model.status])

In [ ]:
# Total Distance
total_distance = pulp.value(model.objective)
print("Total Distance: " , total_distance)

In [ ]:
# Print the results
path=[]
for t in num_steps:
    print(f'step:{t}\n')
    for i in start_locations:
         for j in end_locations:
                if (xVar[(i,j,t)].varValue == 1) :
                    print(f"TRAVEL FROM {i}  TO {j}")
                    
                    if i not in path:
                        path.append(i)
                    if j not in path:
                        path.append(j)

In [ ]:
# Print Results
for t in num_steps: 
    print('Stop number: ' + str(t))
    print('Travel time(mins): ' + str(dVar[t].varValue))
    
    for i in start_locations: 
        for j in end_locations: 
            
            if xVar[(i,j,t)].varValue > 0.0:
                print('\tLeft from location: ' + str(i))
                print('\tArrived at location: ' + str(j))
                print('\tWait time: ' + str(wVar[(j,t)].varValue))
#                 print('\tSum of wait times: ' + str(zVar[t].varValue))
total_wait_time = sum([wVar[(j,t)].varValue for j in end_locations for t in num_steps if wVar[(j,t)].varValue != None])
print('Total wait: ' + str(total_wait_time))
wait_time = str(total_wait_time / len(customers))              
print('Average wait: ' + wait_time)

In [ ]:
route_=[]
for index,r in enumerate(path):
    lat=df_regions.loc[df_regions['name'] == r]['latitude'].values[0]
    log=df_regions.loc[df_regions['name'] == r]['longitude'].values[0]
    route_.append((lat,log))

In [ ]:
# Load the GeoJSON data for Toronto from a URL
url = 'https://raw.githubusercontent.com/jasonicarter/toronto-geojson/master/toronto_crs84.geojson'
response = requests.get(url)
toronto_geojson = response.json()

# Create a map of Toronto
m = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# Add the GeoJSON data to the map as a layer
folium.GeoJson(toronto_geojson).add_to(m)

# Define the route as a list of (lat, lon) coordinates
route = route_

# Add the route to the map as a PolyLine
folium.PolyLine(route, color='red').add_to(m)

# Add numbered markers for each location to the map
for i, location in enumerate(route):
    folium.Marker(location=location, icon=folium.Icon(color='blue'), 
                  popup=str(i+1)).add_to(m)

# Display the map
m


## Sensitivity

### Data Preparation

In [ ]:
# Route A or B
route = 'B'

In [ ]:
starting_location = 'Downtown Toronto (Rosedale)'
average_velocity = 40 # Km/Hr
average_wait = 5 # Mins

# Convert to Km/Min
velocity_min = average_velocity / 60

In [ ]:
# Route variables
test_W = range(0, 120, 5)

if route == 'A':
    print('Route A')
    df = df_ordersA
else:
    print('Route B')
    df = df_ordersB

In [ ]:
# Get start and end locations
start_locations = list(df['restaurant'].unique()) + list(df['customer'].unique())
end_locations = start_locations.copy()

start_locations = list(dict.fromkeys(start_locations))
end_locations = list(dict.fromkeys(end_locations))

if starting_location in start_locations:
    start_locations.remove(starting_location)
    start_locations.append(starting_location)
else:
    start_locations.append(starting_location)


In [ ]:
order_locations = list(df['restaurant']) + list(df['customer'])

In [ ]:
# Calculate the number of stops 
num_steps = np.arange(1, len(end_locations)+1).tolist()

In [ ]:
# Get unique Restraurants and Customers
restaurants = df['restaurant'].unique().tolist()
customers = df['customer'].unique().tolist()

In [ ]:
order_df = pd.DataFrame({'Restaurant':[], 'Customer':[]})

for index, row in df.iterrows():
    order_df.loc[index, 'Restaurant'] = row["restaurant"]
    order_df.loc[index, 'Customer'] = row["customer"]

order_df

In [ ]:
travel_df = pd.DataFrame({'Start':[], 'End': [], 'Distance': [], 'Time': []})


for i in start_locations:
    for j in end_locations: 
        if i == j:
            new_row_data = {'Start': i, 'End': j, 'Distance': 0, 'Time': 0}
            new_row = pd.DataFrame(new_row_data, index=[0])
            travel_df = pd.concat([travel_df, new_row], ignore_index=True)
        else:
            d_temp = float(df_distances[(df_distances['origin'] == i) & 
                                                          (df_distances['destination'] == j)]['distance'])
            t_temp = d_temp/average_velocity * 60
            new_row_data = {'Start': i, 'End': j, 'Distance': d_temp, 'Time': t_temp}
            new_row = pd.DataFrame(new_row_data, index=[0])
            travel_df = pd.concat([travel_df, new_row], ignore_index=True)

travel_df.head()

In [ ]:
df['estimated availability'] = pd.to_datetime(df['estimated availability'])
df['Converted Availability'] = (df['estimated availability'] - df['estimated availability'].min()).dt.total_seconds() / 60

In [ ]:
df

In [ ]:
def one_driver_within_W(W):
    
    print("W:", W)

    model = LpProblem(name = 'Part2_Model', sense = LpMinimize)

    xVar = LpVariable.dict('x', (start_locations, end_locations, num_steps), cat = LpBinary)
    # zVar = LpVariable.dict('z', num_steps, lowBound = 0.0, cat = LpContinuous)
    dVar = LpVariable.dict('d', [0] + num_steps, lowBound = -9999.0, cat = LpContinuous)
    aVar = LpVariable.dict('a', [0] + num_steps, lowBound = -9999.0, cat = LpContinuous)
    wVar = LpVariable.dict('w', (start_locations, num_steps), lowBound = 0.0, cat = LpContinuous)
#     vVar = LpVariable.dict('w', (start_locations, end_locations), lowBound = 0.0, cat = LpContinuous)

    obj = lpSum([travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0] * xVar[(i,j,t)] \
                 for i in start_locations for j in end_locations for t in num_steps])
    model += obj

    model += dVar[(0)] <= 0

    def travel_time_lookup(i, j, travel_df):
        return travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Time'].iloc[0]

    # Step time calculation
    for t in num_steps:
        model += aVar[(t)] == dVar[(t-1)] + lpSum([travel_time_lookup(i, j, travel_df) * xVar[(i,j,t)] \
                                                   for i in start_locations for j in restaurants]) + \
                              lpSum([(travel_time_lookup(i, j, travel_df) + 5) * xVar[(i,j,t)] \
                                     for i in start_locations for j in customers])
        model += aVar[(t)] <= dVar[(t)]

    # Driver leaves after the food is ready
    for t in num_steps:
        for i in start_locations:
            for j in restaurants:
                model += dVar[(t)] >= df[df['restaurant'] == j]['Converted Availability'].iloc[0] * xVar[(i,j,t)]

    # Visit restaurant before customer
    for t in num_steps: 
        for j in end_locations: 
            if j in customers: 
                model += pulp.lpSum( [xVar[i,j,t] for i in start_locations]) <= pulp.lpSum([xVar[i, restaurant_customer_match(j, order_df),t_] \
                                                                                            for i in start_locations \
                                                                                            for t_ in num_steps[:t]])

    for idx, row in df.iterrows():
        rst, cus, time = row['restaurant'], row['customer'], row['Converted Availability']
        for t in num_steps:
            model += wVar[(cus, t)] <= 10000000 * lpSum(xVar[(i, cus, t)] for i in start_locations if i != cus)

            model += wVar[(cus, t)] <= aVar[t] - time + 10000000 * (1 - lpSum(xVar[(i, cus, t)] for i in start_locations if i !=cus))
            model += wVar[(cus, t)] >= aVar[t] - time - 10000000 * (1 - lpSum(xVar[(i, cus, t)] for i in start_locations if i !=cus))

    model += W >= lpSum(wVar[(c,t)] for c in customers for t in num_steps)/len(df)

    # Every location receives order once
    for j in end_locations: 
        model += lpSum([xVar[(i,j,t)] for i in start_locations for t in num_steps]) == 1

    # Convervation of flow
    for t in num_steps[:-1]:
        for j in end_locations:
            model += (lpSum([xVar[(i,j,t)] for i in start_locations]) == lpSum([xVar[(j,k,t+1)] for k in end_locations]))

    # First node has an outflow of 1, others 0
    model += lpSum([xVar[(start_locations[-1],j,1)] for j in restaurants]) == 1

    for i in start_locations[:-1]:
        model += lpSum([xVar[(i,j,1)] for j in end_locations]) == 0

    # Solve the model
    model.solve()
    print("Status:", LpStatus[model.status])
    model_status = LpStatus[model.status]
    
    if LpStatus[model.status] == 'Optimal':
    
        total_distance = pulp.value(model.objective)
        print("Total Distance: " , total_distance)

        total_wait_time = sum([wVar[(j,t)].varValue for j in end_locations for t in num_steps if wVar[(j,t)].varValue != None])
        print('Total wait: ' + str(total_wait_time))
        
        avg_wait_time = total_wait_time/len(customers)
        print('Average wait: ' + str(avg_wait_time))
        
    else:
        total_distance = 0
        total_wait_time = 0
        avg_wait_time = 0
    
    print('=======================================')
        
    return {'w': W,
           'model_status': model_status,
           'total_distance': total_distance,
           'total_wait_time': total_wait_time,
            'avg_wait_time': avg_wait_time
           }

In [ ]:
test_W = range(0, 120, 5)
result_dict = {}
total_distance_list = []
avg_wait_time_list = []

for w in test_W:
    result_dict[w] = one_driver_within_W(w)
    total_distance_list.append(result_dict[w]['total_distance'])
    avg_wait_time_list.append(result_dict[w]['avg_wait_time'])
    
# Plot the optimal W Value
plt.plot(test_W, total_distance_list)
plt.xlabel('W Constraints')
plt.ylabel('Total Distance')
plt.title('W Constraints vs Total Distance - Trade-off Curve')
plt.grid(True)

## New metric - Max Wait Time

In [ ]:
def one_driver_within_W_max(W):
    
    print("W:", W)

    model = LpProblem(name = 'Part2_Model', sense = LpMinimize)

    xVar = LpVariable.dict('x', (start_locations, end_locations, num_steps), cat = LpBinary)
    # zVar = LpVariable.dict('z', num_steps, lowBound = 0.0, cat = LpContinuous)
    dVar = LpVariable.dict('d', [0] + num_steps, lowBound = -9999.0, cat = LpContinuous)
    aVar = LpVariable.dict('a', [0] + num_steps, lowBound = -9999.0, cat = LpContinuous)
    wVar = LpVariable.dict('w', (start_locations, num_steps), lowBound = 0.0, cat = LpContinuous)
    max_var = pulp.LpVariable("max_var", lowBound=0.0, cat = LpContinuous)
#     vVar = LpVariable.dict('w', (start_locations, end_locations), lowBound = 0.0, cat = LpContinuous)

#     obj = lpSum([travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0] * xVar[(i,j,t)] \
#                  for i in start_locations for j in end_locations for t in num_steps])
    
    obj = max_var
    
    model += obj

    model += dVar[(0)] <= 0

    def travel_time_lookup(i, j, travel_df):
        return travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Time'].iloc[0]

    # Step time calculation
    for t in num_steps:
        model += aVar[(t)] == dVar[(t-1)] + lpSum([travel_time_lookup(i, j, travel_df) * xVar[(i,j,t)] \
                                                   for i in start_locations for j in restaurants]) + \
                              lpSum([(travel_time_lookup(i, j, travel_df) + 5) * xVar[(i,j,t)] \
                                     for i in start_locations for j in customers])
        model += aVar[(t)] <= dVar[(t)]

    # Driver leaves after the food is ready
    for t in num_steps:
        for i in start_locations:
            for j in restaurants:
                model += dVar[(t)] >= df[df['restaurant'] == j]['Converted Availability'].iloc[0] * xVar[(i,j,t)]

    # Visit restaurant before customer
    for t in num_steps: 
        for j in end_locations: 
            if j in customers: 
                model += pulp.lpSum( [xVar[i,j,t] for i in start_locations]) <= pulp.lpSum([xVar[i, restaurant_customer_match(j, order_df),t_] \
                                                                                            for i in start_locations \
                                                                                            for t_ in num_steps[:t]])

    for idx, row in df.iterrows():
        rst, cus, time = row['restaurant'], row['customer'], row['Converted Availability']
        for t in num_steps:
            model += wVar[(cus, t)] <= 10000000 * lpSum(xVar[(i, cus, t)] for i in start_locations if i != cus)

            model += wVar[(cus, t)] <= aVar[t] - time + 10000000 * (1 - lpSum(xVar[(i, cus, t)] for i in start_locations if i !=cus))
            model += wVar[(cus, t)] >= aVar[t] - time - 10000000 * (1 - lpSum(xVar[(i, cus, t)] for i in start_locations if i !=cus))

    model += W >= lpSum(wVar[(c,t)] for c in customers for t in num_steps)/len(df)
    
    for t in num_steps:
        for j in end_locations:
            model += max_var >= wVar[(j, t)]

    # Every location receives order once
    for j in end_locations: 
        model += lpSum([xVar[(i,j,t)] for i in start_locations for t in num_steps]) == 1

    # Convervation of flow
    for t in num_steps[:-1]:
        for j in end_locations:
            model += (lpSum([xVar[(i,j,t)] for i in start_locations]) == lpSum([xVar[(j,k,t+1)] for k in end_locations]))

    # First node has an outflow of 1, others 0
    model += lpSum([xVar[(start_locations[-1],j,1)] for j in restaurants]) == 1

    for i in start_locations[:-1]:
        model += lpSum([xVar[(i,j,1)] for j in end_locations]) == 0

    # Solve the model
    model.solve()
    print("Status:", LpStatus[model.status])
    model_status = LpStatus[model.status]
    
    total_dist_temp = 0
    for i in start_locations:
        for j in end_locations:
            for t in num_steps:
                total_dist_temp += travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0] * xVar[(i,j,t)].varValue
    
    
    if LpStatus[model.status] == 'Optimal':
    
        max_wait_time = pulp.value(model.objective)
        print("Max Wait Time: " , max_wait_time)
        
        print("Total Distance: " , total_dist_temp)

        total_wait_time = sum([wVar[(j,t)].varValue for j in end_locations for t in num_steps if wVar[(j,t)].varValue != None])
        print('Total wait: ' + str(total_wait_time))
        
        avg_wait_time = total_wait_time/len(customers)
        print('Average wait: ' + str(avg_wait_time))
        
    else:
        max_wait_time = 0
        total_wait_time = 0
        avg_wait_time = 0
        total_dist_temp = 0
    
    print('=======================================')
        
    return {'w': W,
           'model_status': model_status,
           'total_dist_temp': total_dist_temp,
           'max_wait_time': max_wait_time,
           'total_wait_time': total_wait_time,
            'avg_wait_time': avg_wait_time
           }

In [ ]:
test_W = range(0, 120, 5)
result_dict = {}
max_wait_time_list = []
avg_wait_time_list = []
total_distance_list = []

for w in test_W:
    result_dict[w] = one_driver_within_W_max(w)
    max_wait_time_list.append(result_dict[w]['max_wait_time'])
    avg_wait_time_list.append(result_dict[w]['avg_wait_time'])
    total_distance_list.append(result_dict[w]['total_dist_temp'])
    
# Plot the optimal W Value
plt.plot(test_W, max_wait_time_list)
plt.xlabel('W Constraints')
plt.ylabel('Max Wait Time')
plt.title('W Constraints vs Max Wait Time - Trade-off Curve')
plt.grid(True)

In [ ]:
# Plot the optimal W Value
plt.plot(test_W, total_distance_list)
plt.xlabel('W Constraints')
plt.ylabel('Total Distance')
plt.title('W Constraints vs Total Distance - Trade-off Curve')
plt.grid(True)

# Part III

## Load Data

In [ ]:
df_distances = pd.read_csv(file_path + 'distances.csv')

df_orders = pd.read_csv(file_path + 'part3_small.csv')
df_drivers = pd.read_csv(file_path + 'part3_drivers.csv')

In [ ]:
df_distances.head()

In [ ]:
df_orders

In [ ]:
df_drivers

## Data Preparation

In [ ]:
# Route variables
W = 10000
df = df_orders.copy()

In [ ]:
# Get start and end locations
start_locations = list(df['restaurant'].unique()) + list(df['customer'].unique())
end_locations = start_locations.copy()

start_locations = list(dict.fromkeys(start_locations))
end_locations = list(dict.fromkeys(end_locations))

In [ ]:
df_drivers

In [ ]:
num_drivers = list(range(1,len(df_drivers)+1))
num_drivers

In [ ]:
driver_origins = df_drivers['start region'].to_list()
driver_origins

In [ ]:
for starting_location in driver_origins:
    
    if starting_location in start_locations:
        start_locations.remove(starting_location)
        start_locations.append(starting_location)
    else:
        start_locations.append(starting_location)


In [ ]:
order_locations = list(df['restaurant']) + list(df['customer'])
order_locations

In [ ]:
# Calculate the number of stops 
num_steps = np.arange(1, len(end_locations)+1).tolist()

In [ ]:
# Get unique Restraurants and Customers
restaurants = df['restaurant'].unique().tolist()
customers = df['customer'].unique().tolist()

In [ ]:
order_df = pd.DataFrame({'Restaurant':[], 'Customer':[]})

for index, row in df.iterrows():
    order_df.loc[index, 'Restaurant'] = row["restaurant"]
    order_df.loc[index, 'Customer'] = row["customer"]

order_df

In [ ]:
travel_df = pd.DataFrame({'Start':[], 'End': [], 'Distance': [], 'Time': []})


for i in start_locations:
    for j in end_locations: 
        if i == j:
            new_row_data = {'Start': i, 'End': j, 'Distance': 0, 'Time': 0}
            new_row = pd.DataFrame(new_row_data, index=[0])
            travel_df = pd.concat([travel_df, new_row], ignore_index=True)
        else:
            d_temp = float(df_distances[(df_distances['origin'] == i) & 
                                                          (df_distances['destination'] == j)]['distance'])
            t_temp = d_temp/average_velocity * 60
            new_row_data = {'Start': i, 'End': j, 'Distance': d_temp, 'Time': t_temp}
            new_row = pd.DataFrame(new_row_data, index=[0])
            travel_df = pd.concat([travel_df, new_row], ignore_index=True)

travel_df.head()

In [ ]:
df['estimated availability'] = pd.to_datetime(df['estimated availability'])
df['Converted Availability'] = (df['estimated availability'] - df['estimated availability'].min()).dt.total_seconds() / 60

In [ ]:
df

## Model

In [ ]:
model = LpProblem(name = 'Part3_Model', sense = LpMinimize)

xVar = LpVariable.dict('x', (start_locations, end_locations, num_steps, num_drivers), cat = LpBinary)
# zVar = LpVariable.dict('z', num_steps, lowBound = 0.0, cat = LpContinuous)
dVar = LpVariable.dict('d', ([0] + num_steps, num_drivers), lowBound = -9999.0, cat = LpContinuous)
aVar = LpVariable.dict('a', ([0] + num_steps, num_drivers), lowBound = -9999.0, cat = LpContinuous)
wVar = LpVariable.dict('w', (start_locations, num_steps, num_drivers), lowBound = 0.0, cat = LpContinuous)
# vVar = LpVariable.dict('w', (start_locations, end_locations), lowBound = 0.0, cat = LpContinuous)

In [ ]:
obj = lpSum([travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0] * xVar[(i,j,t,p)] \
             for i in start_locations for j in end_locations for t in num_steps for p in num_drivers])
model += obj

In [ ]:
for p in num_drivers:
    model += dVar[(0, p)] <= 0

In [ ]:
def travel_time_driver_lookup(i, j, p, travel_df, df_drivers):
    d_temp = float(travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0])
    v_temp = float(df_drivers.loc[p-1, 'velocity'])
    return d_temp/v_temp*60

In [ ]:
# # Step time calculation
# for t in num_steps:
#     model += dVar[(t)] == dVar[(t-1)] + lpSum([travel_time_lookup(i, j, travel_df) * xVar[(i,j,t)] \
#                                                for i in start_locations for j in restaurants]) + \
#                           lpSum([(travel_time_lookup(i, j, travel_df) + 5) * xVar[(i,j,t)] \
#                                  for i in start_locations for j in customers])

In [ ]:
# # Assign time to each travel in vVar
# for i in start_locations:
#     for j in end_locations:
#         model += vVar[(i,j)] == travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Time'].iloc[0]

In [ ]:
# # Driver arrives after the food is ready
# for t in num_steps:
#     for i in start_locations:
#         for j in range(0, int(len(order_locations)/2)):
#             model += dVar[(t)] >= df.loc[j, 'Converted Availability'] * xVar[(i,df.loc[j, 'restaurant'],t)]

In [ ]:
# Step time calculation
for p in num_drivers:
    for t in num_steps:
        model += aVar[(t,p)] == dVar[(t-1,p)] + lpSum([travel_time_driver_lookup(i, j, p, travel_df, df_drivers) * xVar[(i,j,t,p)] \
                                                   for i in start_locations for j in restaurants]) + \
                              lpSum([(travel_time_driver_lookup(i, j, p, travel_df, df_drivers) + 5) * xVar[(i,j,t,p)] \
                                     for i in start_locations for j in customers])
        model += aVar[(t,p)] <= dVar[(t,p)]

In [ ]:
# Driver leaves after the food is ready
for p in num_drivers:
    for t in num_steps:
        for i in start_locations:
            for j in restaurants:
                model += dVar[(t,p)] >= df[df['restaurant'] == j]['Converted Availability'].iloc[0] * xVar[(i,j,t,p)]

In [ ]:
# Visit restaurant before customer
for p in num_drivers:
    for t in num_steps: 
        for j in end_locations: 
            if j in customers: 
                model += pulp.lpSum( [xVar[i,j,t,p] for i in start_locations]) <= pulp.lpSum([xVar[i, restaurant_customer_match(j, order_df),t_,p] \
                                                                                            for i in start_locations \
                                                                                            for t_ in num_steps[:t]])

In [ ]:
# for p in num_drivers:
#     for idx, row in df.iterrows():
#         rst, cus, time = row['restaurant'], row['customer'], row['Converted Availability']
#         for t in num_steps:
#             model += wVar[(cus,t,p)] <= 10000000 * lpSum(xVar[(i,cus,t,p)] for i in start_locations if i != cus)

#             model += wVar[(cus,t,p)] <= aVar[t,p] - time + 10000000 * (1 - lpSum(xVar[(i,cus,t,p)] for i in start_locations if i !=cus))
#             model += wVar[(cus,t,p)] >= aVar[t,p] - time - 10000000 * (1 - lpSum(xVar[(i,cus,t,p)] for i in start_locations if i !=cus))

In [ ]:
for p in num_drivers:
    for idx, row in df.iterrows():
        rst, cus, time = row['restaurant'], row['customer'], row['Converted Availability']
        for t in num_steps:
            model += wVar[(cus,t,p)] <= 10000000 * lpSum(xVar[(i,cus,t,p)] for i in start_locations)

            model += wVar[(cus,t,p)] <= aVar[t,p] - time + 10000000 * (1 - lpSum(xVar[(i,cus,t,p)] for i in start_locations))
            model += wVar[(cus,t,p)] >= aVar[t,p] - time - 10000000 * (1 - lpSum(xVar[(i,cus,t,p)] for i in start_locations))

In [ ]:
model += W >= lpSum(wVar[(c,t,p)] for c in customers for t in num_steps for p in num_drivers)/len(df)

In [ ]:
# Every location receives order once
for j in end_locations: 
    model += lpSum([xVar[(i,j,t,p)] for i in start_locations for t in num_steps for p in num_drivers]) == 1

In [ ]:
for p in num_drivers:
    for t in num_steps:
        model += lpSum([xVar[(i,j,t,p)] for i in start_locations for j in end_locations]) <= 1

In [ ]:
# Convervation of flow
for p in num_drivers:
    for t in num_steps[:-1]:
        for j in end_locations:
            if t == 1:
                model += (lpSum([xVar[(i,j,t,p)] for i in start_locations]) >= lpSum([xVar[(j,k,t+1,p)] for k in end_locations]))
            else:
                model += (lpSum([xVar[(i,j,t,p)] for i in end_locations]) >= lpSum([xVar[(j,k,t+1,p)] for k in end_locations]))
            
#             model += (lpSum([xVar[(i,j,t,p)] for i in start_locations])) >= lpSum([xVar[(i,k,t+1,p)] for i in start_locations for k in end_locations])
#             model += 1000000 * (1 - lpSum([xVar[(i,j,t,p)] for i in start_locations]))+1 >= (lpSum([xVar[(i,k,t+1,p)] for i in start_locations for k in end_locations]))
#             model += (lpSum([xVar[(i,j,t,p)] for i in start_locations])) <=1
#             model += (lpSum([xVar[(m,k,t+1,p)] for k in end_locations for m in start_locations if m!= j])) == 0

            model += (lpSum([xVar[(m,k,t+1,p)] for k in end_locations for m in start_locations if m!= j])) <= 1000000 * (1 - lpSum([xVar[(i,j,t,p)] for i in start_locations]))
            model += (lpSum([xVar[(m,k,t+1,p)] for k in end_locations for m in start_locations if m!= j])) >= - 1000000 * (1 - lpSum([xVar[(i,j,t,p)] for i in start_locations]))
                                                                                                                         

In [ ]:
# # First node has an outflow of 1, others 0
# model += lpSum([xVar[(start_locations[-1],j,1)] for j in restaurants]) == 1

# for i in start_locations[:-1]:
#     model += lpSum([xVar[(i,j,1)] for j in end_locations]) == 0

In [ ]:
# All drivers must have orders
model += (lpSum([xVar[(i,j,1,p)] for i in start_locations for j in end_locations for p in num_drivers])) == len(num_drivers)  

In [ ]:
# Drivers must start from different origins
for i in start_locations[-len(num_drivers):]:
    model += (lpSum([xVar[(i,j,1,p)] for j in end_locations for p in num_drivers])) == 1

In [ ]:
# Each driver can only go to 1 location from their origins in step 1
for p in num_drivers:
    model += (lpSum([xVar[(driver_origins[p-1],j,1,p)] for j in end_locations])) <= 1

In [ ]:
for p in num_drivers:
    for i in start_locations[:-len(num_drivers)]:
        model += (lpSum([xVar[(i,j,1,p)] for j in end_locations])) == 0

## Solve

In [ ]:
# Solve the model
model.solve()
print("Status:", LpStatus[model.status])

In [ ]:
# Total Distance
total_distance = pulp.value(model.objective)
print("Total Distance: " , total_distance)

In [ ]:
# Print the results
path=[]
for p in num_drivers:
    print('Driver number: ' + str(p))
    for t in num_steps:
        print(f'step:{t}\n')
        for i in start_locations:
             for j in end_locations:
                    if (xVar[(i,j,t,p)].varValue == 1) :
                        print(f"TRAVEL FROM {i}  TO {j}")

                        if i not in path:
                            path.append(i)
                        if j not in path:
                            path.append(j)

In [ ]:
route_=[]
for index,r in enumerate(path):
    lat=df_regions.loc[df_regions['name'] == r]['latitude'].values[0]
    log=df_regions.loc[df_regions['name'] == r]['longitude'].values[0]
    route_.append((lat,log))

In [ ]:
import matplotlib.colors as mcolors

In [ ]:
# Load the GeoJSON data for Toronto from a URL
url = 'https://raw.githubusercontent.com/jasonicarter/toronto-geojson/master/toronto_crs84.geojson'
response = requests.get(url)
toronto_geojson = response.json()

# Create a map of Toronto
m = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# Add the GeoJSON data to the map as a layer
folium.GeoJson(toronto_geojson).add_to(m)

# Get a list of colors for the different routes
colors = list(mcolors.TABLEAU_COLORS.values())

# Iterate over the drivers
for driver_num, p in enumerate(num_drivers):
    path = []
    for t in num_steps:
        for i in start_locations:
            for j in end_locations:
                if xVar[(i, j, t, p)].varValue == 1:
                    if i not in path:
                        path.append(i)
                    if j not in path:
                        path.append(j)

    route_ = []
    for index, r in enumerate(path):
        lat = df_regions.loc[df_regions['name'] == r]['latitude'].values[0]
        log = df_regions.loc[df_regions['name'] == r]['longitude'].values[0]
        route_.append((lat, log))

    # Add the route to the map as a PolyLine
    folium.PolyLine(route_, color=colors[driver_num % len(colors)]).add_to(m)

    # Add numbered markers for each location to the map
    for i, location in enumerate(route_):
        folium.Marker(location=location, icon=folium.Icon(color=colors[driver_num % len(colors)]),
                      popup=f"Driver {p}: {i + 1}").add_to(m)

# Display the map
m

In [ ]:
# Print Results
for p in num_drivers:
    print('Driver number: ' + str(p))
    for t in num_steps: 
        print('\tStop number: ' + str(t))
        print('\tTravel time(mins): ' + str(dVar[t,p].varValue))

        for i in start_locations: 
            for j in end_locations: 

                if xVar[(i,j,t,p)].varValue > 0.0:
                    print('\t\tLeft from location: ' + str(i))
                    print('\t\tArrived at location: ' + str(j))
                    print('\t\tWait time: ' + str(wVar[(j,t,p)].varValue))
    #                 print('\tSum of wait times: ' + str(zVar[t].varValue))
total_wait_time = sum([wVar[(j,t,p)].varValue for j in end_locations for t in num_steps for p in num_drivers if wVar[(j,t,p)].varValue != None])
print('Total wait: ' + str(total_wait_time))
# wait_time = str(zVar[t].varValue / len(customers))              
# print('Average wait: ' + wait_time)

In [ ]:
xVar[('Downtown Toronto (Richmond / Adelaide / King)','Downtown Toronto (Central Bay Street)',1,3)].varValue

In [ ]:
for p in num_drivers:
    print("p: " + str(p))
    for t in num_steps[:-1]:
        print("t: " + str(t))
        for j in end_locations:
            print("j: " + str(j))
            print('sum1: ' + str(sum([xVar[(i,j,t,p)].varValue for i in start_locations])))
            print('sum2: ' + str(sum([xVar[(j,k,t+1,p)].varValue for k in end_locations])))
#             print('sum2: ' + str(sum([xVar[(i,k,t+1,p)].varValue for k in end_locations])))
            
    print("===========================================")

## Driver Number Change

## Data Preparation

In [ ]:
# Route variables
W = 10000
df = df_orders.copy()

In [ ]:
# Get start and end locations
start_locations = list(df['restaurant'].unique()) + list(df['customer'].unique())
end_locations = start_locations.copy()

start_locations = list(dict.fromkeys(start_locations))
end_locations = list(dict.fromkeys(end_locations))

In [ ]:
order_locations = list(df['restaurant']) + list(df['customer'])
order_locations

In [ ]:
# Calculate the number of stops 
num_steps = np.arange(1, len(end_locations)+1).tolist()

In [ ]:
# Get unique Restraurants and Customers
restaurants = df['restaurant'].unique().tolist()
customers = df['customer'].unique().tolist()

In [ ]:
order_df = pd.DataFrame({'Restaurant':[], 'Customer':[]})

for index, row in df.iterrows():
    order_df.loc[index, 'Restaurant'] = row["restaurant"]
    order_df.loc[index, 'Customer'] = row["customer"]

order_df

In [ ]:
travel_df = pd.DataFrame({'Start':[], 'End': [], 'Distance': [], 'Time': []})


for i in start_locations:
    for j in end_locations: 
        if i == j:
            new_row_data = {'Start': i, 'End': j, 'Distance': 0, 'Time': 0}
            new_row = pd.DataFrame(new_row_data, index=[0])
            travel_df = pd.concat([travel_df, new_row], ignore_index=True)
        else:
            d_temp = float(df_distances[(df_distances['origin'] == i) & 
                                                          (df_distances['destination'] == j)]['distance'])
            t_temp = d_temp/average_velocity * 60
            new_row_data = {'Start': i, 'End': j, 'Distance': d_temp, 'Time': t_temp}
            new_row = pd.DataFrame(new_row_data, index=[0])
            travel_df = pd.concat([travel_df, new_row], ignore_index=True)

travel_df.head()

In [ ]:
df['estimated availability'] = pd.to_datetime(df['estimated availability'])
df['Converted Availability'] = (df['estimated availability'] - df['estimated availability'].min()).dt.total_seconds() / 60

In [ ]:
df

## Model

In [ ]:
def multiple_driver_with_infinite_W(W, df_drivers):
    
    number_drivers = len(df_drivers)
    num_drivers = list(range(1,len(df_drivers)+1))
    print("Driver Number: " + str(number_drivers))
    driver_origins = df_drivers['start region'].to_list()
    
    for starting_location in driver_origins:
    
        if starting_location in start_locations:
            start_locations.remove(starting_location)
            start_locations.append(starting_location)
        else:
            start_locations.append(starting_location)

    travel_df = pd.DataFrame({'Start':[], 'End': [], 'Distance': [], 'Time': []})

    for i in start_locations:
        for j in end_locations: 
            if i == j:
                new_row_data = {'Start': i, 'End': j, 'Distance': 0, 'Time': 0}
                new_row = pd.DataFrame(new_row_data, index=[0])
                travel_df = pd.concat([travel_df, new_row], ignore_index=True)
            else:
                d_temp = float(df_distances[(df_distances['origin'] == i) & 
                                                              (df_distances['destination'] == j)]['distance'])
                t_temp = d_temp/average_velocity * 60
                new_row_data = {'Start': i, 'End': j, 'Distance': d_temp, 'Time': t_temp}
                new_row = pd.DataFrame(new_row_data, index=[0])
                travel_df = pd.concat([travel_df, new_row], ignore_index=True)

    travel_df.head()
    

    model = LpProblem(name = 'Part3_Model', sense = LpMinimize)

    xVar = LpVariable.dict('x', (start_locations, end_locations, num_steps, num_drivers), cat = LpBinary)
    dVar = LpVariable.dict('d', ([0] + num_steps, num_drivers), lowBound = -9999.0, cat = LpContinuous)
    aVar = LpVariable.dict('a', ([0] + num_steps, num_drivers), lowBound = -9999.0, cat = LpContinuous)
    wVar = LpVariable.dict('w', (start_locations, num_steps, num_drivers), lowBound = 0.0, cat = LpContinuous)

    obj = lpSum([travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0] * xVar[(i,j,t,p)] \
                 for i in start_locations for j in end_locations for t in num_steps for p in num_drivers])
    model += obj

    for p in num_drivers:
        model += dVar[(0, p)] <= 0

    def travel_time_driver_lookup(i, j, p, travel_df, df_drivers):
        d_temp = float(travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0])
        v_temp = float(df_drivers.loc[p-1, 'velocity'])
        return d_temp/v_temp*60

    # Step time calculation
    for p in num_drivers:
        for t in num_steps:
            model += aVar[(t,p)] == dVar[(t-1,p)] + lpSum([travel_time_driver_lookup(i, j, p, travel_df, df_drivers) * xVar[(i,j,t,p)] \
                                                       for i in start_locations for j in restaurants]) + \
                                  lpSum([(travel_time_driver_lookup(i, j, p, travel_df, df_drivers) + 5) * xVar[(i,j,t,p)] \
                                         for i in start_locations for j in customers])
            model += aVar[(t,p)] <= dVar[(t,p)]

    # Driver leaves after the food is ready
    for p in num_drivers:
        for t in num_steps:
            for i in start_locations:
                for j in restaurants:
                    model += dVar[(t,p)] >= df[df['restaurant'] == j]['Converted Availability'].iloc[0] * xVar[(i,j,t,p)]

    # Visit restaurant before customer
    for p in num_drivers:
        for t in num_steps: 
            for j in end_locations: 
                if j in customers: 
                    model += pulp.lpSum( [xVar[i,j,t,p] for i in start_locations]) <= pulp.lpSum([xVar[i, restaurant_customer_match(j, order_df),t_,p] \
                                                                                                for i in start_locations \
                                                                                                for t_ in num_steps[:t]])

    for p in num_drivers:
        for idx, row in df.iterrows():
            rst, cus, time = row['restaurant'], row['customer'], row['Converted Availability']
            for t in num_steps:
                model += wVar[(cus,t,p)] <= 10000000 * lpSum(xVar[(i,cus,t,p)] for i in start_locations)

                model += wVar[(cus,t,p)] <= aVar[t,p] - time + 10000000 * (1 - lpSum(xVar[(i,cus,t,p)] for i in start_locations))
                model += wVar[(cus,t,p)] >= aVar[t,p] - time - 10000000 * (1 - lpSum(xVar[(i,cus,t,p)] for i in start_locations))

    model += W >= lpSum(wVar[(c,t,p)] for c in customers for t in num_steps for p in num_drivers)/len(df)

    # Every location receives order once
    for j in end_locations: 
        model += lpSum([xVar[(i,j,t,p)] for i in start_locations for t in num_steps for p in num_drivers]) == 1

    for p in num_drivers:
        for t in num_steps:
            model += lpSum([xVar[(i,j,t,p)] for i in start_locations for j in end_locations]) <= 1

    # Convervation of flow
    for p in num_drivers:
        for t in num_steps[:-1]:
            for j in end_locations:
                if t == 1:
                    model += (lpSum([xVar[(i,j,t,p)] for i in start_locations]) >= lpSum([xVar[(j,k,t+1,p)] for k in end_locations]))
                else:
                    model += (lpSum([xVar[(i,j,t,p)] for i in end_locations]) >= lpSum([xVar[(j,k,t+1,p)] for k in end_locations]))

                model += (lpSum([xVar[(m,k,t+1,p)] for k in end_locations for m in start_locations if m!= j])) <= 1000000 * (1 - lpSum([xVar[(i,j,t,p)] for i in start_locations]))
                model += (lpSum([xVar[(m,k,t+1,p)] for k in end_locations for m in start_locations if m!= j])) >= - 1000000 * (1 - lpSum([xVar[(i,j,t,p)] for i in start_locations]))


    # All drivers must have orders
    model += (lpSum([xVar[(i,j,1,p)] for i in start_locations for j in end_locations for p in num_drivers])) == len(num_drivers)  

    # Drivers must start from different origins
    for i in start_locations[-len(num_drivers):]:
        model += (lpSum([xVar[(i,j,1,p)] for j in end_locations for p in num_drivers])) == 1

    # Each driver can only go to 1 location from their origins in step 1
    for p in num_drivers:
        model += (lpSum([xVar[(driver_origins[p-1],j,1,p)] for j in end_locations])) <= 1

    for p in num_drivers:
        for i in start_locations[:-len(num_drivers)]:
            model += (lpSum([xVar[(i,j,1,p)] for j in end_locations])) == 0
            
    # Solve the model
    model.solve()
    print("Status:", LpStatus[model.status])
    model_status = LpStatus[model.status]
    
    if LpStatus[model.status] == 'Optimal':
    
        total_distance = pulp.value(model.objective)
        print("Total Distance: " , total_distance)

        total_wait_time = sum([wVar[(j,t,p)].varValue for j in end_locations for t in num_steps for p in num_drivers if wVar[(j,t,p)].varValue != None])
        print('Total wait: ' + str(total_wait_time))
        
        avg_wait_time = total_wait_time/len(customers)
        print('Average wait: ' + str(avg_wait_time))
        
    else:
        total_distance = 0
        total_wait_time = 0
        avg_wait_time = 0
    
    print('=======================================')
        
    return {'number_drivers': number_drivers,
           'model_status': model_status,
           'total_distance': total_distance,
           'total_wait_time': total_wait_time,
            'avg_wait_time': avg_wait_time
           }

In [ ]:
end_locations

In [ ]:
start_locations

In [ ]:
travel_df

In [ ]:
result_list = []
driver_r_list = []

combinations = []
for i in range(1, len(df_drivers) + 1):
    for subset in itertools.combinations(df_drivers.index, i):
        combinations.append(subset)

dataframes = []
for combination in combinations:
    dataframes.append(df_drivers.loc[list(combination)])

for i, df_combination in enumerate(dataframes):
    r = multiple_driver_with_infinite_W(W, df_combination.copy().reset_index(drop=True))
    result_list.append(r)
    driver_r_list.append(df_combination['start region'].tolist())
    

scatter_list = []    
for result in result_list:
    scatter_list.append((result['number_drivers'], result['total_distance'], result['avg_wait_time']))
    
display(result_list)
display(driver_r_list)
    
number_drivers, total_distance, avg_wait_time = zip(*scatter_list)

# Normalize the avg_wait_time for better representation in the plot
normalized_wait_time = [x * 50 for x in avg_wait_time]

# # Create the scatter plot
# plt.scatter(number_drivers, total_distance, s=normalized_wait_time, alpha=0.5)
# plt.xlabel('Number of Drivers')
# plt.ylabel('Total Distance')
# plt.title('Scatter plot of Total Distance vs Number of Drivers')

# # Show the plot
# plt.show()

In [ ]:
# Separate the data into lists
number_drivers = [x[0] for x in scatter_list]
total_distance = [x[1] for x in scatter_list]
avg_wait_time = [x[2] for x in scatter_list]

# Create a scatter plot
for drivers in set(number_drivers):
    mask = [x == drivers for x in number_drivers]
    plt.scatter([total_distance[i] for i in range(len(mask)) if mask[i]],
                [avg_wait_time[i] for i in range(len(mask)) if mask[i]],
                s=[drivers * 50 for i in range(len(mask)) if mask[i]],
                label=f'{drivers} drivers', alpha=0.5)

# Set axis labels and title
plt.xlabel('Total Distance')
plt.ylabel('Average Wait Time')
plt.title('Scatter plot of Total Distance vs Average Wait Time')

# Add a legend
plt.legend(title='Number of drivers')

# Show the plot
plt.show()

In [ ]:
test_W = range(0, 120, 30)
result_dict = {}
total_distance_list = []
avg_wait_time_list = []

for w in test_W:
    print('W is: ' + str(w))
    result_dict[w] = multiple_driver_with_infinite_W(w, df_drivers.copy())
    total_distance_list.append(result_dict[w]['total_distance'])
    avg_wait_time_list.append(result_dict[w]['avg_wait_time'])
    
# Plot the optimal W Value
plt.plot(test_W, total_distance_list)
plt.xlabel('W Constraints')
plt.ylabel('Total Distance')
plt.title('W Constraints vs Total Distance - Trade-off Curve')
plt.grid(True)

## New metric - Max Wait Time

In [ ]:
def multiple_driver_with_infinite_W_max(W, df_drivers):
    
    number_drivers = len(df_drivers)
    num_drivers = list(range(1,len(df_drivers)+1))
    print("Driver Number: " + str(number_drivers))
    driver_origins = df_drivers['start region'].to_list()
    
    for starting_location in driver_origins:
    
        if starting_location in start_locations:
            start_locations.remove(starting_location)
            start_locations.append(starting_location)
        else:
            start_locations.append(starting_location)

    travel_df = pd.DataFrame({'Start':[], 'End': [], 'Distance': [], 'Time': []})

    for i in start_locations:
        for j in end_locations: 
            if i == j:
                new_row_data = {'Start': i, 'End': j, 'Distance': 0, 'Time': 0}
                new_row = pd.DataFrame(new_row_data, index=[0])
                travel_df = pd.concat([travel_df, new_row], ignore_index=True)
            else:
                d_temp = float(df_distances[(df_distances['origin'] == i) & 
                                                              (df_distances['destination'] == j)]['distance'])
                t_temp = d_temp/average_velocity * 60
                new_row_data = {'Start': i, 'End': j, 'Distance': d_temp, 'Time': t_temp}
                new_row = pd.DataFrame(new_row_data, index=[0])
                travel_df = pd.concat([travel_df, new_row], ignore_index=True)

    travel_df.head()
    

    model = LpProblem(name = 'Part3_Model', sense = LpMinimize)

    xVar = LpVariable.dict('x', (start_locations, end_locations, num_steps, num_drivers), cat = LpBinary)
    dVar = LpVariable.dict('d', ([0] + num_steps, num_drivers), lowBound = -9999.0, cat = LpContinuous)
    aVar = LpVariable.dict('a', ([0] + num_steps, num_drivers), lowBound = -9999.0, cat = LpContinuous)
    wVar = LpVariable.dict('w', (start_locations, num_steps, num_drivers), lowBound = 0.0, cat = LpContinuous)
    max_var = pulp.LpVariable("max_var", lowBound=0.0, cat = LpContinuous)
    
    obj = max_var
    
#     obj = lpSum([travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0] * xVar[(i,j,t,p)] \
#                  for i in start_locations for j in end_locations for t in num_steps for p in num_drivers])
    model += obj

    for p in num_drivers:
        model += dVar[(0, p)] <= 0

    def travel_time_driver_lookup(i, j, p, travel_df, df_drivers):
        d_temp = float(travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0])
        v_temp = float(df_drivers.loc[p-1, 'velocity'])
        return d_temp/v_temp*60

    # Step time calculation
    for p in num_drivers:
        for t in num_steps:
            model += aVar[(t,p)] == dVar[(t-1,p)] + lpSum([travel_time_driver_lookup(i, j, p, travel_df, df_drivers) * xVar[(i,j,t,p)] \
                                                       for i in start_locations for j in restaurants]) + \
                                  lpSum([(travel_time_driver_lookup(i, j, p, travel_df, df_drivers) + 5) * xVar[(i,j,t,p)] \
                                         for i in start_locations for j in customers])
            model += aVar[(t,p)] <= dVar[(t,p)]

    # Driver leaves after the food is ready
    for p in num_drivers:
        for t in num_steps:
            for i in start_locations:
                for j in restaurants:
                    model += dVar[(t,p)] >= df[df['restaurant'] == j]['Converted Availability'].iloc[0] * xVar[(i,j,t,p)]

    # Visit restaurant before customer
    for p in num_drivers:
        for t in num_steps: 
            for j in end_locations: 
                if j in customers: 
                    model += pulp.lpSum( [xVar[i,j,t,p] for i in start_locations]) <= pulp.lpSum([xVar[i, restaurant_customer_match(j, order_df),t_,p] \
                                                                                                for i in start_locations \
                                                                                                for t_ in num_steps[:t]])

    for p in num_drivers:
        for idx, row in df.iterrows():
            rst, cus, time = row['restaurant'], row['customer'], row['Converted Availability']
            for t in num_steps:
                model += wVar[(cus,t,p)] <= 10000000 * lpSum(xVar[(i,cus,t,p)] for i in start_locations)

                model += wVar[(cus,t,p)] <= aVar[t,p] - time + 10000000 * (1 - lpSum(xVar[(i,cus,t,p)] for i in start_locations))
                model += wVar[(cus,t,p)] >= aVar[t,p] - time - 10000000 * (1 - lpSum(xVar[(i,cus,t,p)] for i in start_locations))
    
    for t in num_steps:
        for j in end_locations:
            model += max_var >= wVar[(j, t, p)]
            
    model += W >= lpSum(wVar[(c,t,p)] for c in customers for t in num_steps for p in num_drivers)/len(df)

    # Every location receives order once
    for j in end_locations: 
        model += lpSum([xVar[(i,j,t,p)] for i in start_locations for t in num_steps for p in num_drivers]) == 1

    for p in num_drivers:
        for t in num_steps:
            model += lpSum([xVar[(i,j,t,p)] for i in start_locations for j in end_locations]) <= 1

    # Convervation of flow
    for p in num_drivers:
        for t in num_steps[:-1]:
            for j in end_locations:
                if t == 1:
                    model += (lpSum([xVar[(i,j,t,p)] for i in start_locations]) >= lpSum([xVar[(j,k,t+1,p)] for k in end_locations]))
                else:
                    model += (lpSum([xVar[(i,j,t,p)] for i in end_locations]) >= lpSum([xVar[(j,k,t+1,p)] for k in end_locations]))

                model += (lpSum([xVar[(m,k,t+1,p)] for k in end_locations for m in start_locations if m!= j])) <= 1000000 * (1 - lpSum([xVar[(i,j,t,p)] for i in start_locations]))
                model += (lpSum([xVar[(m,k,t+1,p)] for k in end_locations for m in start_locations if m!= j])) >= - 1000000 * (1 - lpSum([xVar[(i,j,t,p)] for i in start_locations]))


    # All drivers must have orders
    model += (lpSum([xVar[(i,j,1,p)] for i in start_locations for j in end_locations for p in num_drivers])) == len(num_drivers)  

    # Drivers must start from different origins
    for i in start_locations[-len(num_drivers):]:
        model += (lpSum([xVar[(i,j,1,p)] for j in end_locations for p in num_drivers])) == 1

    # Each driver can only go to 1 location from their origins in step 1
    for p in num_drivers:
        model += (lpSum([xVar[(driver_origins[p-1],j,1,p)] for j in end_locations])) <= 1

    for p in num_drivers:
        for i in start_locations[:-len(num_drivers)]:
            model += (lpSum([xVar[(i,j,1,p)] for j in end_locations])) == 0
            
    # Solve the model
    model.solve()
    print("Status:", LpStatus[model.status])
    model_status = LpStatus[model.status]
    
    total_dist_temp = 0
    for i in start_locations:
        for j in end_locations:
            for t in num_steps:
                total_dist_temp += travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0] * xVar[(i,j,t)].varValue
    
    
    if LpStatus[model.status] == 'Optimal':
    
        max_wait_time = pulp.value(model.objective)
        print("Max Wait Time: " , max_wait_time)
        
        print("Total Distance: " , total_dist_temp)

        total_wait_time = sum([wVar[(j,t,p)].varValue for j in end_locations for t in num_steps for p in num_drivers if wVar[(j,t,p)].varValue != None])
        print('Total wait: ' + str(total_wait_time))
        
        avg_wait_time = total_wait_time/len(customers)
        print('Average wait: ' + str(avg_wait_time))
        
    else:
        max_wait_time = 0
        total_wait_time = 0
        avg_wait_time = 0
        total_dist_temp = 0
    
    print('=======================================')
        
    return {'number_drivers': number_drivers,
           'model_status': model_status,
           'total_dist_temp': total_dist_temp,
           'max_wait_time': max_wait_time,
           'total_wait_time': total_wait_time,
            'avg_wait_time': avg_wait_time
           }


In [ ]:
result_dict_temp = multiple_driver_with_infinite_W_max(120, df_drivers.copy())
result_dict_temp

# Part IV

## part3_small.csv

## Load Data

In [ ]:
df_distances = pd.read_csv(file_path + 'distances.csv')

df_orders = pd.read_csv(file_path + 'part3_small.csv')
df_drivers = pd.read_csv(file_path + 'part3_drivers.csv')
df_regions = pd.read_csv(file_path + 'regions.csv')

In [ ]:
df_distances.head()

In [ ]:
df_orders

In [ ]:
df_drivers

In [ ]:
df_regions.head()

## Data Preparation

In [ ]:
# Route variables
W = 120
df = df_orders.copy()

In [ ]:
df_merged = df.merge(df_regions, left_on='customer', right_on='name')
df_merged.head()

In [ ]:
df_drivers = df_drivers.merge(df_regions, left_on='start region', right_on='name')
df_drivers.head()

In [ ]:
coordinates = df_merged[['latitude', 'longitude']]
coordinates

In [ ]:
df_cluster_centers = df_drivers[['latitude', 'longitude']]
df_cluster_centers

In [ ]:
# Initialize KMeans with predefined cluster centers
kmeans = KMeans(n_clusters=len(df_cluster_centers), init=df_cluster_centers.values, n_init=1)

# Fit the model and predict cluster labels
df_merged['cluster'] = kmeans.fit_predict(coordinates)
df_merged

In [ ]:
cluster_to_driver = {i: df_drivers.index[i] for i in range(len(df_cluster_centers))}

# Add a 'driver' column to the df_merged dataframe
df_merged['driver'] = df_merged['cluster'].map(cluster_to_driver)

In [ ]:
df_merged

In [ ]:
# Optional: Visualize the results
plt.scatter(df_merged['latitude'], df_merged['longitude'], c=df_merged['cluster'], cmap='viridis')
plt.scatter(df_cluster_centers['latitude'], df_cluster_centers['longitude'], c='red', marker='x')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('KMeans Clustering with Predefined Cluster Centers')
plt.show()

In [ ]:
def one_driver_within_W_P4(W, starting_location, velocity, df, df_distances):
    
    order_quantity = len(df)
    
    # Get start and end locations
    start_locations = list(df['restaurant'].unique()) + list(df['customer'].unique())
    end_locations = start_locations.copy()

    start_locations = list(dict.fromkeys(start_locations))
    end_locations = list(dict.fromkeys(end_locations))

    if starting_location in start_locations:
        start_locations.remove(starting_location)
        start_locations.append(starting_location)
    else:
        start_locations.append(starting_location)   
        
    order_locations = list(df['restaurant']) + list(df['customer'])
    
    # Calculate the number of stops 
    num_steps = np.arange(1, len(end_locations)+1).tolist()
    
    # Get unique Restraurants and Customers
    restaurants = df['restaurant'].unique().tolist()
    customers = df['customer'].unique().tolist()

    order_df = pd.DataFrame({'Restaurant':[], 'Customer':[]})

    for index, row in df.iterrows():
        order_df.loc[index, 'Restaurant'] = row["restaurant"]
        order_df.loc[index, 'Customer'] = row["customer"]
    
    travel_df = pd.DataFrame({'Start':[], 'End': [], 'Distance': [], 'Time': []})

    for i in start_locations:
        for j in end_locations: 
            if i == j:
                new_row_data = {'Start': i, 'End': j, 'Distance': 0, 'Time': 0}
                new_row = pd.DataFrame(new_row_data, index=[0])
                travel_df = pd.concat([travel_df, new_row], ignore_index=True)
            else:
                d_temp = float(df_distances[(df_distances['origin'] == i) & 
                                                              (df_distances['destination'] == j)]['distance'])
                t_temp = d_temp/velocity * 60
                new_row_data = {'Start': i, 'End': j, 'Distance': d_temp, 'Time': t_temp}
                new_row = pd.DataFrame(new_row_data, index=[0])
                travel_df = pd.concat([travel_df, new_row], ignore_index=True)

    df['estimated availability'] = pd.to_datetime(df['estimated availability'])
    df['Converted Availability'] = (df['estimated availability'] - df['estimated availability'].min()).dt.total_seconds() / 60                
                
    ###################

    model = LpProblem(name = 'Part2_Model', sense = LpMinimize)

    xVar = LpVariable.dict('x', (start_locations, end_locations, num_steps), cat = LpBinary)
    # zVar = LpVariable.dict('z', num_steps, lowBound = 0.0, cat = LpContinuous)
    dVar = LpVariable.dict('d', [0] + num_steps, lowBound = -9999.0, cat = LpContinuous)
    aVar = LpVariable.dict('a', [0] + num_steps, lowBound = -9999.0, cat = LpContinuous)
    wVar = LpVariable.dict('w', (start_locations, num_steps), lowBound = 0.0, cat = LpContinuous)
#     vVar = LpVariable.dict('w', (start_locations, end_locations), lowBound = 0.0, cat = LpContinuous)

    obj = lpSum([travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Distance'].iloc[0] * xVar[(i,j,t)] \
                 for i in start_locations for j in end_locations for t in num_steps])
    model += obj

    model += dVar[(0)] <= 0

    def travel_time_lookup(i, j, travel_df):
        return travel_df[(travel_df['Start']==i) & (travel_df['End']==j)]['Time'].iloc[0]

    # Step time calculation
    for t in num_steps:
        model += aVar[(t)] == dVar[(t-1)] + lpSum([travel_time_lookup(i, j, travel_df) * xVar[(i,j,t)] \
                                                   for i in start_locations for j in restaurants]) + \
                              lpSum([(travel_time_lookup(i, j, travel_df) + 5) * xVar[(i,j,t)] \
                                     for i in start_locations for j in customers])
        model += aVar[(t)] <= dVar[(t)]

    # Driver leaves after the food is ready
    for t in num_steps:
        for i in start_locations:
            for j in restaurants:
                model += dVar[(t)] >= df[df['restaurant'] == j]['Converted Availability'].iloc[0] * xVar[(i,j,t)]

    # Visit restaurant before customer
    for t in num_steps: 
        for j in end_locations: 
            if j in customers: 
                model += pulp.lpSum( [xVar[i,j,t] for i in start_locations]) <= pulp.lpSum([xVar[i, restaurant_customer_match(j, order_df),t_] \
                                                                                            for i in start_locations \
                                                                                            for t_ in num_steps[:t]])

    for idx, row in df.iterrows():
        rst, cus, time = row['restaurant'], row['customer'], row['Converted Availability']
        for t in num_steps:
            model += wVar[(cus, t)] <= 10000000 * lpSum(xVar[(i, cus, t)] for i in start_locations if i != cus)

            model += wVar[(cus, t)] <= aVar[t] - time + 10000000 * (1 - lpSum(xVar[(i, cus, t)] for i in start_locations if i !=cus))
            model += wVar[(cus, t)] >= aVar[t] - time - 10000000 * (1 - lpSum(xVar[(i, cus, t)] for i in start_locations if i !=cus))

    model += W >= lpSum(wVar[(c,t)] for c in customers for t in num_steps)/len(df)

    # Every location receives order once
    for j in end_locations: 
        model += lpSum([xVar[(i,j,t)] for i in start_locations for t in num_steps]) == 1

    # Convervation of flow
    for t in num_steps[:-1]:
        for j in end_locations:
            model += (lpSum([xVar[(i,j,t)] for i in start_locations]) == lpSum([xVar[(j,k,t+1)] for k in end_locations]))

    # First node has an outflow of 1, others 0
    model += lpSum([xVar[(start_locations[-1],j,1)] for j in restaurants]) == 1

    for i in start_locations[:-1]:
        model += lpSum([xVar[(i,j,1)] for j in end_locations]) == 0

    # Solve the model
    model.solve()
    print("Status:", LpStatus[model.status])
    model_status = LpStatus[model.status]
    
    if LpStatus[model.status] == 'Optimal':
    
        total_distance = pulp.value(model.objective)
        print("Total Distance: " , total_distance)

        total_wait_time = sum([wVar[(j,t)].varValue for j in end_locations for t in num_steps if wVar[(j,t)].varValue != None])
        print('Total wait: ' + str(total_wait_time))
        
        avg_wait_time = total_wait_time/len(customers)
        print('Average wait: ' + str(avg_wait_time))
        
    else:
        total_distance = 0
        total_wait_time = 0
        avg_wait_time = 0
        
    
    path_summary_df = pd.DataFrame({'Step Number':[], 'Departure Time (min)':[], 'Leaving': [], 
                                    'Arriving': [], 'Wait Time': []})

    for t in num_steps: 
#         print('Stop number: ' + str(t))
        path_summary_df.loc[t, 'Step Number'] = t
#         print('Travel time(mins): ' + str(dVar[t].varValue))
        path_summary_df.loc[t, 'Departure Time (min)'] = dVar[t].varValue

        for i in start_locations: 
            for j in end_locations: 

                if xVar[(i,j,t)].varValue > 0.0:
                    path_summary_df.loc[t, 'Leaving'] = i
#                     print('\tLeft from location: ' + str(i))
                    path_summary_df.loc[t, 'Arriving'] = j
#                     print('\tArrived at location: ' + str(j))
                    path_summary_df.loc[t, 'Wait Time'] = wVar[(j,t)].varValue
#                     print('\tWait time: ' + str(wVar[(j,t)].varValue))
                    
    
    print('=======================================')
        
    display(path_summary_df)

    return {'order_quantity': order_quantity,
           'model_status': model_status,
           'total_distance': total_distance,
           'total_wait_time': total_wait_time,
            'avg_wait_time': avg_wait_time
           }, total_distance, total_wait_time, order_quantity

In [ ]:
# Group the dataframe by the 'cluster' column
grouped_df = df_merged.groupby('cluster')

# Create a list of dataframes for each cluster, resetting the index
dfs_by_cluster = [group.reset_index(drop=True) for _, group in grouped_df]

r_list = []
total_distance_list = []
total_wait_list = []
number_orders_list = []

# Print the dataframes in the list
for i, cluster_df in enumerate(dfs_by_cluster):
    driver = cluster_df.loc[0, 'driver']
    
    start_region = df_drivers.loc[driver, 'start region']
    velocity = df_drivers.loc[driver, 'velocity']
    cluster_num = cluster_df.iloc[0,-1]
    
    print("Cluster: " + str(cluster_num) + "\n" + "###############################")
    
    r_dict, total_distance, total_wait, number_orders = one_driver_within_W_P4(W, start_region, velocity, 
                                                                       cluster_df.copy(), df_distances.copy())
    r_list.append(r_dict)
    total_distance_list.append(total_distance)
    total_wait_list.append(total_wait)
    number_orders_list.append(number_orders)
    

In [ ]:
final_total_distance = sum(total_distance_list)
final_average_wait = sum(total_wait_list)/sum(number_orders_list)

print("Total distance: " + str(final_total_distance))
print("Average wait time: " + str(final_average_wait))

## part4_large.csv with part4_drivers.csv

## Load Data

In [ ]:
df_distances = pd.read_csv(file_path + 'distances.csv')

df_orders = pd.read_csv(file_path + 'part4_large.csv')
df_drivers = pd.read_csv(file_path + 'part4_drivers.csv')
df_regions = pd.read_csv(file_path + 'regions.csv')

In [ ]:
df_distances.head()

In [ ]:
df_orders.head()

In [ ]:
df_drivers.head()

In [ ]:
df_regions.head()

## Data Preparation

In [ ]:
# Route variables
W = 120
df = df_orders.copy()

In [ ]:
df_merged = df.merge(df_regions, left_on='customer', right_on='name')
df_merged.head()

In [ ]:
df_drivers = df_drivers.merge(df_regions, left_on='start region', right_on='name')
df_drivers.head()

In [ ]:
coordinates = df_merged[['latitude', 'longitude']]
coordinates

In [ ]:
df_cluster_centers = df_drivers[['latitude', 'longitude']]
df_cluster_centers

In [ ]:
# Initialize KMeans with predefined cluster centers
kmeans = KMeans(n_clusters=len(df_cluster_centers), init=df_cluster_centers.values, n_init=1)

# Fit the model and predict cluster labels
df_merged['cluster'] = kmeans.fit_predict(coordinates)
df_merged

In [ ]:
cluster_to_driver = {i: df_drivers.index[i] for i in range(len(df_cluster_centers))}

# Add a 'driver' column to the df_merged dataframe
df_merged['driver'] = df_merged['cluster'].map(cluster_to_driver)

In [ ]:
df_merged

In [ ]:
# Optional: Visualize the results
plt.scatter(df_merged['latitude'], df_merged['longitude'], c=df_merged['cluster'], cmap='viridis')
plt.scatter(df_cluster_centers['latitude'], df_cluster_centers['longitude'], c='red', marker='x')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('KMeans Clustering with Predefined Cluster Centers')
plt.show()

In [ ]:
# Group the dataframe by the 'cluster' column
grouped_df = df_merged.groupby('cluster')

# Create a list of dataframes for each cluster, resetting the index
dfs_by_cluster = [group.reset_index(drop=True) for _, group in grouped_df]

r_list = []
total_distance_list = []
total_wait_list = []
number_orders_list = []

# Print the dataframes in the list
for i, cluster_df in enumerate(dfs_by_cluster):
    driver = cluster_df.loc[0, 'driver']
    
    start_region = df_drivers.loc[driver, 'start region']
    velocity = df_drivers.loc[driver, 'velocity']
    cluster_num = cluster_df.iloc[0,-1]
    
    print("Cluster: " + str(cluster_num) + "\n" + "###############################")
    
    r_dict, total_distance, total_wait, number_orders = one_driver_within_W_P4(W, start_region, velocity, 
                                                                       cluster_df.copy(), df_distances.copy())
    r_list.append(r_dict)
    total_distance_list.append(total_distance)
    total_wait_list.append(total_wait)
    number_orders_list.append(number_orders)
    

In [ ]:
final_total_distance = sum(total_distance_list)
final_average_wait = sum(total_wait_list)/sum(number_orders_list)

print("Total distance: " + str(final_total_distance))
print("Average wait time: " + str(final_average_wait))

In [ ]:
dfs_by_cluster[0]